In [1]:
!pip install --upgrade scikit-learn==0.24.2

     |████████████████████████████████| 22.3 MB 4.6 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.3 which is incompatible.
hypertools 0.7.0 requires scikit-learn!=0.22,<0.24,>=0.19.1, but you have scikit-learn 0.24.2 which is incompatible.


Old version of sklearn. 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
import sklearn
sklearn.__version__

'0.24.2'

This is an old version of sklearn. Need to update manually

Selected Features:

In [4]:
ordinal_features = ["Street", "Alley", "YearBuilt","YearRemodAdd", "ExterQual", "ExterCond", "CentralAir", "GarageYrBlt", "PavedDrive", "YrSold", "MoSold", "FireplaceQu", "HeatingQC", "KitchenQual", "PoolQC"]
numerical_features = ["MSSubClass", "LotFrontage", "LotArea", "OverallQual", "OverallCond", "1stFlrSF", "2ndFlrSF", "FullBath", "HalfBath", "TotalBsmtSF"]
categorical_features = ["MSZoning",  "Heating", "BldgType", "HouseStyle","Foundation", "GarageType","GarageFinish", "SaleCondition", "SaleType"]


In [5]:
df_train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [6]:
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [7]:
df_test.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

recode features before transforming

In [8]:
def recode_likert(df, var_name):
    condition_map =  {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1, "NA": 0}
    return df.loc[:, var_name].map(condition_map)

df_test.loc[:, "HeatingQC"] = recode_likert(df_test, "HeatingQC")
df_test.loc[:, "FireplaceQu"] = recode_likert(df_test, "FireplaceQu")
df_test.loc[:, "KitchenQual"] = recode_likert(df_test, "KitchenQual")
df_test.loc[:, "PoolQC"] = recode_likert(df_test, "PoolQC")
df_test.loc[:, "BsmtCond"] = recode_likert(df_test, "BsmtCond")

df_train.loc[:, "HeatingQC"] = recode_likert(df_train, "HeatingQC")
df_train.loc[:, "FireplaceQu"] = recode_likert(df_train, "FireplaceQu")
df_train.loc[:, "KitchenQual"] = recode_likert(df_train, "KitchenQual")
df_train.loc[:, "PoolQC"] = recode_likert(df_train, "PoolQC")
df_train.loc[:, "BsmtCond"] = recode_likert(df_train, "BsmtCond")

LandSlope = {"Gtl": 0, "Mod": 1, "Sev": 2}

df_test.loc[:, "LandSlope"] = df_test.loc[:, "LandSlope"].map(LandSlope)
df_test.loc[:, "2ndFlrSF"].fillna(value= 0, inplace=True)

df_train.loc[:, "LandSlope"] = df_train.loc[:, "LandSlope"].map(LandSlope)
df_train.loc[:, "2ndFlrSF"].fillna(value= 0, inplace=True)

having errors in the pipeline because I'm mixing strings and ints. Need to change the ordinal vars to strings?

In [9]:
df_train.loc[:, ordinal_features] = df_train.loc[:, ordinal_features].applymap(str)
df_test.loc[:, ordinal_features] = df_test.loc[:, ordinal_features].applymap(str) 

Nope looks like it's in categorical vars.

In [10]:
df_train.loc[:, categorical_features] = df_train.loc[:, categorical_features ].applymap(str)
df_test.loc[:, categorical_features] = df_test.loc[:, categorical_features ].applymap(str)

not in categorical vars. So let's print out each var and it's type

So ordinal is still mixed.

still not converting from int to str

In [11]:
df_train.loc[:, ordinal_features] = df_train.loc[:, ordinal_features].astype(str)
df_test.loc[:, ordinal_features] = df_test.loc[:, ordinal_features].astype(str) 

In [12]:
df_train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold            object
YrSold            object
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

Assign after updating data

In [13]:
train_x = df_train[numerical_features + ordinal_features + categorical_features]
train_y = df_train["SalePrice"]
test_x = df_test[numerical_features + ordinal_features + categorical_features]


In [18]:

numerical_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy= "median")),
    ("enc", RobustScaler())
])


ordinal_pipeline = Pipeline([ 
    ("imp", SimpleImputer(strategy = "constant", fill_value=-1.0)),
    ("enc", OrdinalEncoder(handle_unknown = "use_encoded_value", unknown_value = -1))
])

categorical_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy= "most_frequent")),
    ("enc", OneHotEncoder(sparse=True, handle_unknown = "ignore"))
])


pre_pipe = ColumnTransformer([
    ("cat_pre", categorical_pipeline, categorical_features),
    ("ord_pre", ordinal_pipeline, ordinal_features),
    ("num_pre", numerical_pipeline, numerical_features)
])


In [19]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(loss = 'lad')

In [20]:
full_pipe = Pipeline([
    ("pre", pre_pipe),
    ("model", model)
])

In [21]:
full_pipe.fit(train_x, train_y)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('cat_pre',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('enc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['MSZoning', 'Heating',
                                                   'BldgType', 'HouseStyle',
                                                   'Foundation', 'GarageType',
                                                   'GarageFinish',
                                                   'SaleCondition',
                                                   'SaleType']),
                                                 ('ord_pre',
                                                  Pipelin

In [ ]:
train_x

In [ ]:
test_x

In [22]:
y_hat = full_pipe.predict(test_x)
y_hat

array([128247.93670896, 155537.34197615, 170175.0409524 , ...,
       175284.51472031, 123041.88948353, 212073.81375859])

In [ ]:
test_x.iloc[0, :]

In [ ]:
test_x.iloc[:,1]